<a href="https://colab.research.google.com/github/ambicapvs/ambica_info5731_spring2021/blob/main/datacleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
dataframe = pd.read_csv('/content/vaccinetweets.csv',sep=',',engine ='python', header=0)
dataframe

,tweetid,moderna,pfizer
0,0,"⚠️ Moins de Moderna, plus de Pfizer... l'appro...","⚠️ Moins de Moderna, plus de Pfizer... l'appro..."
1,1,@quartzpuerta @MrVanGriffin @AlanSarapa I real...,@AlexBerenson @pfizer @bmj_latest Link To This...
2,2,"@marcetravisany QUEDAN PFIZER, MODERNA, SPUTNI...",@escobar_riera Dr. Tuve covid en enero antes d...
3,3,@GrayAdalind As you seem to have an interest i...,"@cupidzclozet me - in fact, I am 20 days past ..."
4,4,@lazaroperez768 Lázaro cariño mío..\r\nBuena ...,@quartzpuerta @MrVanGriffin @AlanSarapa I real...
...,...,...,...
59995,9995,@BadAstronomer Moderna #2 body-slammed me but ...,"Covid, Pfizer identifica vaccini falsi anche i..."
59996,9996,Πόσο διαρκεί η προστασία από τα εμβόλια Pfizer...,@AndrewZung @Gimzar @richard680news Why wouldn...
59997,9997,@AmberB_SD Moderna. Everyone is different thou...,@Azael18A La estrategia fue la misma que con l...
59998,9998,FIRST MODERNA SHOT DOWN!\r\n\r\nAhhhhhhh it’s ...,48 hours since my 2nd Pfizer shot and I have n...


In [5]:
import requests
from textblob import Word
import nltk
nltk.download('wordnet')
nltk.download('punkt')
from textblob import TextBlob
from nltk.stem import PorterStemmer


def CleanData(dfcolumn):
  linksremoved = dfcolumn.replace(to_replace=r'^https?:\/\/.*[\r\n]*',value='',regex=True)
  punctuationremoved = linksremoved.str.replace('[^\w\s]','')
  spclcharsremoved = punctuationremoved.str.replace('[^A-Za-z0-9 ]+','')
  numsremoved = spclcharsremoved.str.replace('[^A-Za-z ]+','')
  data = requests.get("https://gist.githubusercontent.com/sebleier/554280/raw/7e0e4a1ce04c2bb7bd41089c9821dbcf6d0c786c/NLTK's%2520list%2520of%2520english%2520stopwords")
  a = data.text
  stop = a.split()
  stopwordsremoved = numsremoved.apply(lambda x: " ".join(x for x in x.split() if x not in stop))
  lowercased = stopwordsremoved.apply(lambda x: " ".join(x.lower() for x in x.split()))
  tokenized = lowercased.apply(lambda x: TextBlob(x).words)
  st = PorterStemmer()
  stemmed = tokenized.apply(lambda x: " ".join([st.stem(word) for word in x]))
  #Lemmatization
  lemmatized = stemmed.apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
  return lemmatized

modn = CleanData(dataframe['moderna'])
pfizr = CleanData(dataframe['pfizer'])
dataframe['moderna_cleaned'] = modn
dataframe['pfizer_cleaned'] = pfizr

dataframe

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,tweetid,moderna,pfizer,moderna_cleaned,pfizer_cleaned
0,0,"⚠️ Moins de Moderna, plus de Pfizer... l'appro...","⚠️ Moins de Moderna, plus de Pfizer... l'appro...",moin de moderna plu de pfizer lapprovisionn en...,moin de moderna plu de pfizer lapprovisionn en...
1,1,@quartzpuerta @MrVanGriffin @AlanSarapa I real...,@AlexBerenson @pfizer @bmj_latest Link To This...,quartzpuerta mrvangriffin alansarapa i realli ...,alexberenson pfizer bmjlatest link to thi bmj ...
2,2,"@marcetravisany QUEDAN PFIZER, MODERNA, SPUTNI...",@escobar_riera Dr. Tuve covid en enero antes d...,marcetravisani quedan pfizer moderna sputnik y...,escobarriera dr tuve covid en enero ant de aye...
3,3,@GrayAdalind As you seem to have an interest i...,"@cupidzclozet me - in fact, I am 20 days past ...",grayadalind a seem interest testimoni got mode...,cupidzclozet fact i day past nd shot pfizer
4,4,@lazaroperez768 Lázaro cariño mío..\r\nBuena ...,@quartzpuerta @MrVanGriffin @AlanSarapa I real...,lazaroperez lzaro cario mobuena tardeni duda c...,quartzpuerta mrvangriffin alansarapa i realli ...
...,...,...,...,...,...
59995,9995,@BadAstronomer Moderna #2 body-slammed me but ...,"Covid, Pfizer identifica vaccini falsi anche i...",badastronom moderna bodyslam i would show mode...,covid pfizer identifica vaccini falsi anch eur...
59996,9996,Πόσο διαρκεί η προστασία από τα εμβόλια Pfizer...,@AndrewZung @Gimzar @richard680news Why wouldn...,pfizer moderna johnson amp johnsonhttpstcoabph...,andrewzung gimzar richardnew whi wouldnt pfize...
59997,9997,@AmberB_SD Moderna. Everyone is different thou...,@Azael18A La estrategia fue la misma que con l...,amberbsd moderna everyon differ though my bodi...,azaela la estrategia fue la misma que con la p...
59998,9998,FIRST MODERNA SHOT DOWN!\r\n\r\nAhhhhhhh it’s ...,48 hours since my 2nd Pfizer shot and I have n...,first moderna shot downahhhhhhh weeeeird get v...,hour sinc nd pfizer shot i experienc advers si...
